In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime as tm
import seaborn as sns
t = time.time()

In [2]:
def convert2unix(time):
    if type(time) == type('hey'):
        time = tm.strptime(time,'%m/%d/%y %H:%M:%S %p').timestamp()
    elif type(time) == tm:
        time = time.timestamp()
    elif type(time) == float:
        pass
    elif type(time) == pd._libs.tslibs.timestamps.Timestamp:
        time = time.timestamp()
    else:
        print('NONE OF THE ABOVE')
        print(type(time))

    return time


In [3]:
### $ Clear all variables

### $ Filename imported by field engineer
filename = 'C:/Users/sambe/Desktop/Spring 2021/M E 266KP/Given Data/chevron_test_template_short.xlsx'#testP-1605&A New dataset 2.23.21.xlsx'

allvars = ['Inlet Flowrate',
 'Inlet Pressure',
 'Outlet Pressure',
 'Inlet Temperature',
 'Column Level',
 'STD API',
 'Pump Driver Vibration',
 'Pump Vibration',
 'Sister Pump Driver Vibration',
 'Sister Pump Vibration']
### $ Convert to pandas data frame
df = pd.read_excel(filename, sheet_name = 'Time-Series Data')
faildata = pd.read_excel(filename, sheet_name = 'Failure Mode & Date')
print(f'Elapsed time is {(time.time() - t)/60} minutes')
a = df.columns
availvars = []
for word in a:
    if word.startswith('Unnamed'):
        pass
    else:
        availvars.append(word)
### $ Get the number of rows and columns in the file (# of covariates and sampling will probably be different)
number_rows = len(df.index)
number_cols = len(df.columns)

### $ Create empty data frames with the three points per covariate (value, times, good/bad)
variables = pd.DataFrame([])
times = pd.DataFrame()
goodbad = pd.DataFrame()


Elapsed time is 0.5579967737197876 minutes


In [4]:
df.head()

,Unnamed: 0,Inlet Flowrate,Unnamed: 2,Unnamed: 3,Unnamed: 4,Inlet Pressure,Unnamed: 6,Unnamed: 7,Unnamed: 8,Outlet Pressure,...,Pump Vibration,Unnamed: 30,Unnamed: 31,Unnamed: 32,Sister Pump Driver Vibration,Unnamed: 34,Unnamed: 35,Unnamed: 36,Sister Pump Vibration,Unnamed: 38
0,Pen Name:,16FI226,Units: MBPD,NaN,Pen Name:,16pi2115,Units: PSIG,NaN,Pen Name:,16pi2115,...,16ve1133,Units: IN/SEC,NaN,EMPTY 1,EMPTY 2,EMPTY 3,NaN,EMPTY 1,EMPTY 2,EMPTY 3
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5600.16,2018-02-20 15:02:00,GOOD,NaN,44.1016,2018-02-20 15:02:00,GOOD,NaN,2040,2018-02-20 15:02:00,...,2018-02-20 15:02:00,GOOD,NaN,0.02,2018-02-20 15:02:00,GOOD,NaN,0.19,2018-02-20 15:02:00,GOOD
3,5614.88,2018-02-20 15:04:00,GOOD,NaN,44.1328,2018-02-20 15:04:00,GOOD,NaN,2040.94,2018-02-20 15:04:00,...,2018-02-20 15:04:00,GOOD,NaN,0,2018-02-20 15:04:00,GOOD,NaN,0.17,2018-02-20 15:04:00,GOOD
4,5640.78,2018-02-20 15:06:00,GOOD,NaN,44.125,2018-02-20 15:06:00,GOOD,NaN,2042.81,2018-02-20 15:06:00,...,2018-02-20 15:06:00,GOOD,NaN,0.02,2018-02-20 15:06:00,GOOD,NaN,0.23,2018-02-20 15:06:00,GOOD


In [5]:
df.columns = df.iloc[0].fillna('Unnamed')
df.drop(0, inplace = True)
df.reset_index(drop = True, inplace = True)
df

,Pen Name:,16FI226,Units: MBPD,Unnamed,Pen Name:,16pi2115,Units: PSIG,Unnamed,Pen Name:,16pi2115,...,16ve1133,Units: IN/SEC,Unnamed,EMPTY 1,EMPTY 2,EMPTY 3,Unnamed,EMPTY 1,EMPTY 2,EMPTY 3
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5600.16,2018-02-20 15:02:00,GOOD,NaN,44.1016,2018-02-20 15:02:00,GOOD,NaN,2040,2018-02-20 15:02:00,...,2018-02-20 15:02:00,GOOD,NaN,0.02,2018-02-20 15:02:00,GOOD,NaN,0.19,2018-02-20 15:02:00,GOOD
2,5614.88,2018-02-20 15:04:00,GOOD,NaN,44.1328,2018-02-20 15:04:00,GOOD,NaN,2040.94,2018-02-20 15:04:00,...,2018-02-20 15:04:00,GOOD,NaN,0,2018-02-20 15:04:00,GOOD,NaN,0.17,2018-02-20 15:04:00,GOOD
3,5640.78,2018-02-20 15:06:00,GOOD,NaN,44.125,2018-02-20 15:06:00,GOOD,NaN,2042.81,2018-02-20 15:06:00,...,2018-02-20 15:06:00,GOOD,NaN,0.02,2018-02-20 15:06:00,GOOD,NaN,0.23,2018-02-20 15:06:00,GOOD
4,5600.66,2018-02-20 15:08:00,GOOD,NaN,44.125,2018-02-20 15:08:00,GOOD,NaN,2042.81,2018-02-20 15:08:00,...,2018-02-20 15:08:00,GOOD,NaN,0.02,2018-02-20 15:08:00,GOOD,NaN,0,2018-02-20 15:08:00,GOOD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207,5987.62,2018-02-21 00:22:00,GOOD,NaN,42.8203,2018-02-21 00:22:00,GOOD,NaN,1925.16,2018-02-21 00:22:00,...,2018-02-21 00:22:00,GOOD,NaN,0.02,2018-02-21 00:22:00,GOOD,NaN,0.37,2018-02-21 00:22:00,GOOD
208,6047.54,2018-02-21 00:24:00,GOOD,NaN,42.7422,2018-02-21 00:24:00,GOOD,NaN,1924.84,2018-02-21 00:24:00,...,2018-02-21 00:24:00,GOOD,NaN,0.02,2018-02-21 00:24:00,GOOD,NaN,0,2018-02-21 00:24:00,GOOD
209,5991.68,2018-02-21 00:26:00,GOOD,NaN,42.7812,2018-02-21 00:26:00,GOOD,NaN,1924.53,2018-02-21 00:26:00,...,2018-02-21 00:26:00,GOOD,NaN,0.02,2018-02-21 00:26:00,GOOD,NaN,0.24,2018-02-21 00:26:00,GOOD
210,6005.39,2018-02-21 00:28:00,GOOD,NaN,42.7891,2018-02-21 00:28:00,GOOD,NaN,1924.53,2018-02-21 00:28:00,...,2018-02-21 00:28:00,GOOD,NaN,0.02,2018-02-21 00:28:00,GOOD,NaN,0.25,2018-02-21 00:28:00,GOOD


In [6]:
df.columns

Index(['Pen Name:', '16FI226', 'Units: MBPD', 'Unnamed', 'Pen Name:',
       '16pi2115', 'Units: PSIG', 'Unnamed', 'Pen Name:', '16pi2115',
       'Units: PSIG', 'Unnamed', 'EMPTY 1', 'EMPTY 2', 'EMPTY 3', 'Unnamed',
       'EMPTY 1', 'EMPTY 2', 'EMPTY 3', 'Unnamed', 'EMPTY 1', 'EMPTY 2',
       'EMPTY 3', 'Unnamed', 'Pen Name:', '16ve1132', 'Units: IN/SEC',
       'Unnamed', 'Pen Name:', '16ve1133', 'Units: IN/SEC', 'Unnamed',
       'EMPTY 1', 'EMPTY 2', 'EMPTY 3', 'Unnamed', 'EMPTY 1', 'EMPTY 2',
       'EMPTY 3'],
      dtype='object', name=0)

In [7]:
### $ Sort out whole excel file to place the column in the right data frame (value, times, good/bad)
### $ Relabeled with Variable i names
### $ Iterates over the number of columns
t = time.time()
missingcols = []
for i in range(number_cols):
    ### $ Puts column title into str
    str = df.columns[i]
    ### $ if searches for a title that starts with 'Pen Name'
    if str.startswith('Pen Name'):
        ### $ takes variables from column and puts in separate 'variables' dataframe
        s = df.iloc[1:,i]
        variables = pd.concat([variables,s.rename('Value')], axis = 1)
    ### $ elif searches for a title that starts with 'Units'
    elif str.startswith('Units'):
        ### $ takes variables from column and puts in separate 'goodbad' dataframe
        s = df.iloc[1:,i]
        goodbad = pd.concat([goodbad,s.rename('GOOD/BAD')],axis = 1)
    ### $ This is for the blank columns in between variable columns
    elif str.startswith('Unnamed'):
        ### $ Skips over empty columns
        continue
    elif str.startswith('EMPTY 1'):
        variables = pd.concat([variables,pd.DataFrame([np.nan])],axis = 1)
    elif str.startswith('EMPTY 2'):
        times = pd.concat([times,pd.DataFrame([np.nan])],axis = 1)
        missingcols.append(int((i-1)/4))
    elif str.startswith('EMPTY 3'):
        goodbad = pd.concat([goodbad,pd.DataFrame([np.nan])],axis = 1)
    ### $ This is an else because the pen names vary and will be different for every pump/variable
    else:
        ### $ takes variables from column and puts in separate 'times' dataframe
        s = df.iloc[1:,i].apply(convert2unix)
        times = pd.concat([times,s.rename('Time')], axis = 1)
print(f'Elapsed time is {(time.time() - t)/60} minutes')
times.columns = allvars
goodbad.columns = allvars
variables.columns = allvars

Elapsed time is 0.0007827679316202799 minutes


In [8]:
times.drop(0,inplace = True)
times.reset_index(drop = True, inplace = True)
variables.drop(0,inplace = True)
variables.reset_index(drop = True, inplace = True)
goodbad.drop(0,inplace = True)
goodbad.reset_index(drop = True, inplace = True)

In [9]:
missingcols

[3, 4, 5, 8, 9]

for j in range(len(times.columns)):
    for i in range(len(times.iloc[:,j].dropna())):
        a = times.iloc[i,j]
        times.iloc[i,j] = a.timestamp()

In [10]:
### $ interpolates for BAD Values
### $ Excludes last row because you can't interpolate a machine being on or off
t = time.time()
for j in range(len(variables.columns)):
    for i in range(len(variables.iloc[:,j].dropna())):
        str = goodbad.iloc[i,j]
        ### $ If the value is bad and the value has no prior value to interpolate from, just use next good value
        ### $ Needs improvement: could back interpolate and take care of case in which there are consecutive bad values     
        if str.startswith('B'):# and i == 0:
            variables.iloc[i,j] = np.nan
#            goodbad.iloc[i,j] = 'INT'
        ### $ Else if bad, then interpolate between the two good points
#        elif str.startswith('B'):
#            t1 = times.iloc[i-1,j]
#            t2 = times.iloc[i,j]
#            t3 = times.iloc[i+1,j]
#            a = variables.iloc[i-1,j]
#            c = variables.iloc[i+1,j]
#            variables.iloc[i,j] = ((t2-t1)/(t3-t1))*(c-a)+a
#            goodbad.iloc[i,j] = 'INT'
print(f'Elapsed time is {(time.time() - t)/60} minutes')

Elapsed time is 0.0006482481956481933 minutes


In [11]:
### $ Align times via binning
### $ kappa designates the grouping size
### $ Any timestamp outside of the bin will be group later



### $ Sets to dataframes: windows and shutters. windows is the number row that is being looked at for that variable. shutters is end of the length of that variable
### $ column titles designate which variable is being investigated
windows = pd.DataFrame()
shutters = pd.DataFrame()
for j in range(len(variables.columns)):
    w = pd.DataFrame([0], columns = [f'{j}'])
    windows = pd.concat([windows, w], axis = 1)
    temp = variables.iloc[:,j]
    s = pd.DataFrame([len(temp.dropna())], columns = [f'{j}'])
    shutters = pd.concat([shutters, s], axis = 1)

In [12]:
binsize = 60*2 #seconds

### $ abstime is the merged and grouped times of all the variables
abstime = pd.DataFrame([[0]], columns = ['Time'])
### $ vars is the merged and grouped variables
vars = pd.DataFrame([])
### $ This creates a data frame with the exact number of columns as the number of variables
for j in range(len(variables.columns)):
    a = ['NaN']
    vars[f'Variable {j}'] = a
#vars.drop(columns = [0], inplace = True)
### $ counter just used for progress, I guess
#counter = 
### $ Flag1 designates that this is the very first time through the while statement. This flag is used for formatting the data
Flag1 = 0
### $ This iterates until each individual variable has its respective bin


times.drop(0, inplace = True)
times.dropna(axis = 1, how = 'all', inplace = True)
times

In [13]:
times

,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Inlet Temperature,Column Level,STD API,Pump Driver Vibration,Pump Vibration,Sister Pump Driver Vibration,Sister Pump Vibration
0,1.519161e+09,1.519161e+09,1.519161e+09,NaN,NaN,NaN,1.519161e+09,1.519161e+09,NaN,NaN
1,1.519161e+09,1.519161e+09,1.519161e+09,NaN,NaN,NaN,1.519161e+09,1.519161e+09,NaN,NaN
2,1.519161e+09,1.519161e+09,1.519161e+09,NaN,NaN,NaN,1.519161e+09,1.519161e+09,NaN,NaN
3,1.519161e+09,1.519161e+09,1.519161e+09,NaN,NaN,NaN,1.519161e+09,1.519161e+09,NaN,NaN
4,1.519161e+09,1.519161e+09,1.519161e+09,NaN,NaN,NaN,1.519161e+09,1.519161e+09,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
206,1.519194e+09,1.519194e+09,1.519194e+09,NaN,NaN,NaN,1.519194e+09,1.519194e+09,NaN,NaN
207,1.519194e+09,1.519194e+09,1.519194e+09,NaN,NaN,NaN,1.519194e+09,1.519194e+09,NaN,NaN
208,1.519194e+09,1.519194e+09,1.519194e+09,NaN,NaN,NaN,1.519194e+09,1.519194e+09,NaN,NaN
209,1.519194e+09,1.519194e+09,1.519194e+09,NaN,NaN,NaN,1.519194e+09,1.519194e+09,NaN,NaN


In [14]:
windows

,0,1,2,3,4,5,6,7,8,9
0,0,0,0,0,0,0,0,0,0,0


In [15]:
t = time.time()
while max(shutters.iloc[0,:]-windows.iloc[0,:]) != 0:
    timenow = []
    ### $ timenow is built and designates the exact timestamps that are to be compared and binned, if possible 
    for k in range(len(variables.columns)):
        if k in missingcols:
            pass
        else:
            timenow.append(times.iloc[windows.iloc[0,k],k])
    ### $ now is just the earliest times in the timenow group          
    now = min(timenow)
    now = now - (now % binsize)
    ### $ next we round now to the nearest bin value ### $### $### $ Really want to improve this later
    #now = now.round(freq = f'{bin}T')
    ### $ a is a pd.DataFrame created to append to the abstime dataframe
    a = pd.DataFrame([[now]], columns = ['Time'])
    abstime = abstime.append(a, ignore_index = True)
    ### $ Flag0 designates that this is the first time through that specific row. This is also for formatting and allows the rest of the variables to be placed on the right row.
    Flag0 = 0
   
    for l in range(len(variables.columns)):
        ### $ Makes sure that the time is within that bin, if not the program skips any input into the times or variables dataframes
        if times.iloc[windows.iloc[0,l],l] - now <= binsize:
            ### $ If this is the first time through the row
            if Flag0 == 0:
                ### $ Appends a to a brand new row of the variables dataframe. This in affect adds a row to the dataframe
                a = pd.DataFrame([variables.iloc[windows.iloc[0,l],l]], columns = [f'Variable {l}'])
                vars = vars.append(a, ignore_index = True)
                ### $ If this is the first time throught the while statement
                if Flag1 == 0:
                    ### $ Formatting
#                    vars = vars.drop(columns = [0])
                    vars = vars.drop([0])
                    Flag1 = 1
                ### $ If the time is within the bin, it assigns that variable the bin (group) time
                times.iloc[windows.iloc[0,l],l] = now
                ### $ If the time and variable make it into the bin, the window moves to the next value, one row down
                windows.iloc[0,l] +=1
                Flag0 = 1
                #counter += 1
                #print(counter)
            ### $ If this is not the first time through the row, the program does not have to append another row, but can insert the value right into the cell
            else:
                vars.iloc[windows.iloc[0,l],l] = variables.iloc[windows.iloc[0,l],l]
                times.iloc[windows.iloc[0,l],l] = now
                ### $ Still moves the window because it was binned
                windows.iloc[0,l] +=1
        ### $ If its not in the bin, ignore it.
        else:
            continue
print(f'Elapsed time is {(time.time() - t)/60} minutes')


Elapsed time is 0.015844651063283286 minutes


In [16]:
abstime

,Time
0,0.000000e+00
1,1.519161e+09
2,1.519161e+09
3,1.519161e+09
4,1.519161e+09
...,...
207,1.519194e+09
208,1.519194e+09
209,1.519194e+09
210,1.519194e+09


t = time.time()
### $ Formatting
abstime = abstime.drop([0])
### $ Also formatting
vars.index += 1
a = pd.DataFrame()
abstimes = pd.DataFrame([])
var = pd.DataFrame([], columns = allvars)
for i in range(len(abstime)-1):
    if abstime.iloc[i,0] == abstime.iloc[i+1,0]:
        b = pd.DataFrame([vars.iloc[i,:]], columns = vars.columns.values)
        a = pd.concat([a, b])
    else:
        try:

            t = pd.DataFrame([abstime.iloc[i,0]], columns = ['Time'])
            z = pd.DataFrame()
            for j in range(len(vars.columns)):
                r = pd.DataFrame([a.iloc[:,j].mean()])
                z = pd.concat([z,r], axis = 1)
            z.columns = allvars
            abstimes = pd.concat([abstimes, t], ignore_index = True)
            var = pd.concat([var,z], ignore_index = True)
            a = pd.DataFrame()
        except IndexError:
                continue
print(f'Elapsed time is {(time.time() - t)/60} minutes')

In [17]:
vars

,Variable 0,Variable 1,Variable 2,Variable 3,Variable 4,Variable 5,Variable 6,Variable 7,Variable 8,Variable 9
0,5600.16,44.1016,2040,NaN,NaN,NaN,0.0674609,0.0221094,NaN,NaN
1,5614.88,44.1328,2040.94,NaN,NaN,NaN,0.0772656,0.0239844,NaN,NaN
2,5640.78,44.125,2042.81,NaN,NaN,NaN,0.0671875,0.0215625,NaN,NaN
3,5600.66,44.125,2042.81,NaN,NaN,NaN,0.0788281,0.0212109,NaN,NaN
4,5650.43,44.2109,2042.03,NaN,NaN,NaN,0.0696484,0.0241016,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
206,5987.62,42.8203,1925.16,NaN,NaN,NaN,0.0842187,0.0195703,NaN,NaN
207,6047.54,42.7422,1924.84,NaN,NaN,NaN,0.0632812,0.0205469,NaN,NaN
208,5991.68,42.7812,1924.53,NaN,NaN,NaN,0.084375,0.0244531,NaN,NaN
209,6005.39,42.7891,1924.53,NaN,NaN,NaN,0.0673047,0.0194531,NaN,NaN


In [27]:
#for i in range(len(abstimes)):
#    abstimes.iloc[i,0] = pd.to_datetime(abstimes.iloc[i,0], unit = 's')

#for i in range(len(abstime)):
#    abstime.iloc[i,0] = tm.strptime(abstime.iloc[i,0],'%Y-%m-%d %H:%M:%S')
a = ['Time']
for s in allvars:
    a.append(s)

fixed = pd.concat([abstime,vars], axis = 1)
fixed.columns = a

#fixed.set_index('Time', inplace = True)
fixed.to_excel('interpolatedvar.xlsx')


In [28]:
fixed

,Time,Inlet Flowrate,Inlet Pressure,Outlet Pressure,Inlet Temperature,Column Level,STD API,Pump Driver Vibration,Pump Vibration,Sister Pump Driver Vibration,Sister Pump Vibration
0,0.000000e+00,5600.16,44.1016,2040,NaN,NaN,NaN,0.0674609,0.0221094,NaN,NaN
1,1.519161e+09,5614.88,44.1328,2040.94,NaN,NaN,NaN,0.0772656,0.0239844,NaN,NaN
2,1.519161e+09,5640.78,44.125,2042.81,NaN,NaN,NaN,0.0671875,0.0215625,NaN,NaN
3,1.519161e+09,5600.66,44.125,2042.81,NaN,NaN,NaN,0.0788281,0.0212109,NaN,NaN
4,1.519161e+09,5650.43,44.2109,2042.03,NaN,NaN,NaN,0.0696484,0.0241016,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
207,1.519194e+09,6047.54,42.7422,1924.84,NaN,NaN,NaN,0.0632812,0.0205469,NaN,NaN
208,1.519194e+09,5991.68,42.7812,1924.53,NaN,NaN,NaN,0.084375,0.0244531,NaN,NaN
209,1.519194e+09,6005.39,42.7891,1924.53,NaN,NaN,NaN,0.0673047,0.0194531,NaN,NaN
210,1.519194e+09,6064.8,42.7578,1924.84,NaN,NaN,NaN,0.0802734,0.0233203,NaN,NaN


In [23]:
faildata

,Date,Mode
0,2021-02-23 15:50:00,Motor_Distorted
1,2020-11-02 04:15:00,Broken_Valve
2,2020-03-25 21:48:50,Valve_Alignment
3,2020-06-18 09:32:00,Broken_Impeller
4,2020-07-10 15:52:00,Leaking_Valve
5,2020-08-30 01:14:00,Cracked_Seal


In [ ]:
olddata = pd.read_csv('C:/Users/sambe/Desktop/Spring 2021/M E 266KP/SP Data/Compiled_data.csv')

In [ ]:
oldcols = ['Time','X Vib','Suction','Discharge','Flow Rate','Y Vib','Status','Condition']

In [ ]:
olddata.columns = oldcols

In [ ]:
olddata = olddata[olddata['Status'] == 'On']

In [ ]:
olddata.to_csv('On_Compiled_Data.csv')